In [2]:
import numpy as np
import math
import os
import gdal
import xarray as xr
from skimage.measure import block_reduce
from ftplib import FTP
import subprocess
import rioxarray
import time as T
import datetime

class SGLI_L2_Downloader:
    
    def __init__ (self, User_ID , ProductName, Ver, Date , Tile , Obrit ,ParaVer, Target_path):
        self.User_ID = User_ID
        self.ProductName = ProductName
        self.Ver = Ver
        self.Date = Date
        self.Tile = Tile
        self.Obrit = Obrit
        self.ParaVer = ParaVer
        self.Target_path = Target_path
        
        
    def path (self):
        return '/standard/GCOM-C/GCOM-C.SGLI/L2.LAND.{}/{}/{}/{}/{}'.format(self.ProductName, self.Ver, self.Date[0:4], self.Date[4:6], self.Date[6:8])

    def filename (self):
        return 'GC1SG1_{}{}01D_T{}_L2SG_{}Q_{}00{}.h5'.format(self.Date, self.Obrit, self.Tile, self.ProductName, self.Ver, self.ParaVer)
    
    def login (self):
        ftp_addr = 'ftp.gportal.jaxa.jp'
        f = FTP(ftp_addr)
        f.login(self.User_ID,'anonymous')
        
        return f
    
    def download (self, f):
        remote_filepath = self.path()
        f.cwd(remote_filepath)
        list=f.nlst()
        
        bufsize=1024
        if self.filename() in list:
            data = open ('{}{}'.format(self.Target_path, self.filename()), 'wb')
            filename = 'RETR ' + self.filename()
            f.retrbinary(filename,data.write,bufsize)
            return 'y'
        else:
            return 'n'
    
    def logout (self, f):
        f.quit()
    
def reporjection_GEO(filename,Band,Target_path):
    
    #print(path)
    subprocess.check_output('/media/liwei/Data/GCOM-C_RSRF/SGLI_geo_map_linux.exe {} -d Geometry_data/{} -o {} -s 7.5'.format(filename,Band,Target_path[:-1]), shell=True) 
    
def mkdir(path):
    folder = os.path.exists(path) 
    if not folder:
        os.makedirs(path)
        
    
def SGLI_2_AHI_GEO(filepath):
    geotiff_da = rioxarray.open_rasterio(filepath)
    data = geotiff_da.interp(x=lon_y,y=lat_x,method="nearest")
    data = np.where(data != -32768, data ,np.nan)
    data = data * 0.01
    data = data.reshape(40,40)
    data = block_reduce(data, block_size=(2,2), func=np.nanmean)
    return data

def month_size(month,YYYY):
    big_month=['01','03','05','07','08','10','12']
    small_month=['04','06','09','11']
    if month in big_month:
        s = 31
    elif month in small_month:
        s = 30
    elif month == '02':
        if (int(YYYY) % 4 == 0 and int(YYYY) % 100 != 0) or int(YYYY) % 400 == 0:
            s = 29
        else:
            s = 28
    return s

def get_site_range(lat,lon):
    crow = int(60 - lat) / 0.04
    ccol = int(lon - 85) / 0.04
    return int(crow-2),int(crow+2),int(ccol-2),int(ccol+2)

def download_tile(date,save_tile_path,tile):
    g = SGLI_L2_Downloader(
        'galiwei ' ,
        'RSRF', 
        '3', 
         date , 
         tile, 
        'D' , 
        '1' ,
         save_tile_path
        )
    f = g.login()
    
    if os.path.exists(save_tile_path + g.filename()):
        return g.filename()
    
    else :
        rc = g.download(f)
        if rc == 'n':
            g.logout(f)
            return 'n'
        else:
            g.logout(f)
            return g.filename()
    
def download_AHI(target_path , date, time, angle):
    
    date1 = date[0:6]
    date2 = date
    AHI_date = date2 + time
    sza_file_name = '{}.sun.zth.fld.4km.bin.bz2'.format(AHI_date)
    saa_file_name = '{}.sun.azm.fld.4km.bin.bz2'.format(AHI_date)
    
    
    if angle == 'sza':
        if os.path.exists(target_path + sza_file_name[:-4]):
            return sza_file_name[:-4]
    
        else:
            ftp = FTP()
            ftp.connect('hmwr829gr.cr.chiba-u.ac.jp', 21)
            ftp.login()
            path = '/gridded/FD/V20190123/{}/4KM/{}/'.format(date1,date2)
            ftp.cwd(path)
            remote_list=ftp.nlst()
            bufsize=1024*1024
            if sza_file_name in remote_list:
                data = open(target_path + sza_file_name,'wb')
                filename='RETR '+ sza_file_name
                ftp.retrbinary(filename,data.write,bufsize)
                ftp.quit()

                return sza_file_name[:-4]
            else:
#                 print('AHI NO DATA')
                return None
        
    elif angle == 'saa':
        if os.path.exists(target_path + saa_file_name[:-4]):
            return saa_file_name[:-4]
    
        else:
            ftp = FTP()
            ftp.connect('hmwr829gr.cr.chiba-u.ac.jp', 21)
            ftp.login()
            path = '/gridded/FD/V20190123/{}/4KM/{}/'.format(date1,date2)
            ftp.cwd(path)
            remote_list=ftp.nlst()
            bufsize=1024*1024
            if saa_file_name in remote_list:
                data = open(target_path + saa_file_name,'wb')
                filename='RETR '+ saa_file_name
                ftp.retrbinary(filename,data.write,bufsize)
                ftp.quit()
                return saa_file_name[:-4]
            else:
#                 print('AHI NO DATA')
                return None
    
def Decompression(t,fn):
    if os.path.exists('{}.bz2'.format(t + fn)):
        p = subprocess.Popen('lbzip2 -d {}.bz2'.format(t + fn),shell=True)
        p.communicate()
        
def ahi_lon_timeoffset(lon): 
    lon_interval = 15 
    UTC_e_lon = lon_interval / 2 
    timeoffset = math.ceil((lon - UTC_e_lon) / lon_interval) 
    return timeoffset 

In [22]:
YYYY='2021'

# MM = ['11','12']
if YYYY == '2020':
    MM = ['07','08','09','10','11','12']
else:
    MM = ['01','02','03','04','05','06']

# MM = ['05','06']
DD=['01','02','03','04','05','06','07','08','09','10',\
    '11','12','13','14','15','16','17','18','19','20',\
    '21','22','23','24','25','26','27','28','29','30',\
    '31']
HH=['09','10','11']
MIN=['00','10','20','30','40','50']

band = ['Solar_azimuth_PL', 'Solar_zenith_PL','Sensor_zenith_PL', 'Sensor_azimuth_PL']

# site_name = 'test 2'
# site = [51.2,157,1] # ROI 1
# tile = '0327'

# site_name = 'test 1'
# site = [16.75,96.5] # ROI 1
# tile = '0727'


# site_name = 'ROI 1'
# site = [-38.5,177] # ROI 1
# tile = '1231'

# site_name = 'ROI 2'
# site = [-44.5,170.5] # ROI 2
# tile = '1330'


# site_name = 'ROI 3'
# site = [27,96.5] # ROI 3
# tile = '0626'# ROI 3


site_name = 'ROI 4'
site = [32.5,96.5]
tile = '0526'


# site_name = 'ROI 5'
# site = [39.5,107]
# tile = '0526'

# site_name = 'ROI 6'
# site = [46,114]
# tile = '0425'

jet = ahi_lon_timeoffset(site[1])

site_name = 'ROI 4'
site = [32.5,96.5]
tile = '0526'


MATCH_RESULT = []
#Save path
reporjection_file = '/media/liwei/Data/GCOM-C_RSRF_Valid/reporjection/'
tile_file = '/media/liwei/Data/GCOM-C_RSRF_Valid/tile/'
AHI_SAA_t = '/media/liwei/Data/AHI_Angle/Solar_azimuth_angle/'
AHI_SZA_t = '/media/liwei/Data/AHI_Angle/Solar_zenith_angle/'



urow,drow,lcol,rcol = get_site_range(site[0],site[1])
lat_x=np.linspace(site[0]+0.05,site[0]-0.0475,40)
lon_y=np.linspace(site[1]-0.05,site[1]+0.0475,40)


# AHI VZA/VAA Load
with open('/media/liwei/Data/AHI_Angle/AHI_VZA.dat','rb') as fp:
    AHI_VZA = np.frombuffer(fp.read()).reshape(3000,3000)[urow:drow,lcol:rcol]
AHI_VZA = np.nanmean(AHI_VZA)
with open('/media/liwei/Data/AHI_Angle/AHI_VAA.dat','rb') as fp:
    AHI_VAA = np.frombuffer(fp.read()).reshape(3000,3000)[urow:drow,lcol:rcol]
AHI_VAA = np.nanmean(AHI_VAA)

In [ ]:
for m in range(len(MM)):
    s = month_size(MM[m],YYYY)
    for d in range(s):
        date = YYYY+MM[m]+DD[d]
        print(date)
        
#         SGLI file name
        SGLI_VZA_filename = 'GC1SG1_{}D01D_T{}_L2SG_RSRFQ_3001_Sensor_zenith_PL.tif'.format(date,tile)
        SGLI_VAA_filename = 'GC1SG1_{}D01D_T{}_L2SG_RSRFQ_3001_Sensor_azimuth_PL.tif'.format(date,tile)
        SGLI_SZA_filename = 'GC1SG1_{}D01D_T{}_L2SG_RSRFQ_3001_Solar_zenith_PL.tif'.format(date,tile)
        SGLI_SAA_filename = 'GC1SG1_{}D01D_T{}_L2SG_RSRFQ_3001_Solar_azimuth_PL.tif'.format(date,tile)
        
        
#         Reporjection file exist
        if not os.path.exists(reporjection_file + SGLI_VZA_filename):
            
#             print('Start SGLI Download and Reporjection.')
            fn = download_tile(date,tile_file,tile)
            if fn == 'n':
                continue
            for b in band:
                reporjection_GEO( tile_file+fn , b , reporjection_file)
#             print('Reporjection Finsihed!')
             
        if os.path.exists(reporjection_file + SGLI_VZA_filename):       
            SGLI_VZA_PL = SGLI_2_AHI_GEO(reporjection_file + SGLI_VZA_filename)
            SGLI_VZA_PL = np.nanmean(SGLI_VZA_PL)
            VZA_Matching_PL = abs(AHI_VZA - SGLI_VZA_PL)
            
            
            
#             VZA_Matching_PL = 1 - math.cos(math.radians(max(AHI_VZA,SGLI_VZA_PL))) / math.cos(math.radians(min(AHI_VZA,SGLI_VZA_PL)))
    #         print("AHI_VZA:",AHI_VZA,"SGLI_VZA:",SGLI_VZA_PL)
#             print(SGLI_VZA_PL)   
            
#             if VZA_Matching_PL <= 0.02 and VZA_Matching_PL!= 0:
#                 print (date)
                
#             print("AHI_VZA:",AHI_VZA,"SGLI_VZA:",SGLI_VZA_PL)
        
            if VZA_Matching_PL <= 10:
    #             For loop for min
#                 print("AHI_VZA:",AHI_VZA,"SGLI_VZA:",SGLI_VZA_PL)

                for h in HH:
                    for minu in MIN:
                        date_old = datetime.datetime.strptime('{}-{}-{} {}:{}'.format(YYYY,MM[m],DD[d],h,minu), '%Y-%m-%d %H:%M')
                        delta = datetime.timedelta(hours = jet)
                        
                        if jet <= 9:
                            d_new = date_old - delta
                        else :
                            d_new = date_old + delta
                        date_new = d_new.strftime('%Y') + d_new.strftime('%m') + d_new.strftime('%d')
                        TIME = d_new.strftime('%H') + d_new.strftime('%M')


    #                     Download CEReS AHI Solar angle 
                        AHI_SAA_file_name = download_AHI(AHI_SAA_t , date_new, TIME, 'saa')
                        if AHI_SAA_file_name != None:

                            Decompression(AHI_SAA_t , AHI_SAA_file_name)
        #                     AHI SAA Load
                            with open(AHI_SAA_t + AHI_SAA_file_name, 'rb') as fp:
                                AHI_SAA = np.frombuffer(fp.read(), dtype='>f4').reshape(3000,3000)[urow:drow,lcol:rcol]
                            AHI_SAA = np.mean(AHI_SAA)
                            
        #                     SGLI SAA/VAA Load

                            SGLI_VAA_PL = SGLI_2_AHI_GEO(reporjection_file + SGLI_VAA_filename)
                            SGLI_SAA_PL = SGLI_2_AHI_GEO(reporjection_file + SGLI_SAA_filename)
                            SGLI_VAA_PL = np.where(SGLI_VAA_PL<0,SGLI_VAA_PL+360,SGLI_VAA_PL)
                            SGLI_SAA_PL = np.where(SGLI_SAA_PL<0,SGLI_SAA_PL+360,SGLI_SAA_PL)
                            SGLI_VAA_PL = np.nanmean(SGLI_VAA_PL)
                            SGLI_SAA_PL = np.nanmean(SGLI_SAA_PL)
        
        #                     RAA Calulation
                            SGLI_RAA_PL = abs(SGLI_SAA_PL - SGLI_VAA_PL)
                            if SGLI_RAA_PL > 180:
                                SGLI_RAA_PL = 360 - SGLI_RAA_PL

                            AHI_RAA = abs(AHI_SAA - AHI_VAA)
                            if AHI_RAA > 180:
                                AHI_RAA = 360 - AHI_RAA                        
        #                     print(AHI_RAA,SGLI_RAA_PL)



                            RAA_Matching_PL = abs(SGLI_RAA_PL - AHI_RAA)
#                             print(RAA_Matching_PL)
        #                     
                            if RAA_Matching_PL < 15:
#                                 print(SGLI_RAA_PL , AHI_RAA)

        #                         Load AHI/SGLI SZA
                                AHI_SZA_file_name = download_AHI(AHI_SZA_t , date_new, TIME, 'sza')
                                if AHI_SZA_file_name != None:

                                    Decompression(AHI_SZA_t,AHI_SZA_file_name)
                                    with open(AHI_SZA_t + AHI_SZA_file_name, 'rb') as fp:
                                        AHI_SZA = np.frombuffer(fp.read(),dtype='>f4').reshape(3000,3000)[urow:drow,lcol:rcol]
                                    AHI_SZA = np.mean(AHI_SZA)
                                    SGLI_SZA_PL = np.nanmean(SGLI_2_AHI_GEO(reporjection_file + SGLI_SZA_filename))
#                                     print(AHI_SZA,SGLI_SZA_PL)
#                                     SZA_Matching_PL = 1 - math.cos(math.radians(max(AHI_SZA,SGLI_SZA_PL))) / math.cos(math.radians(min(AHI_SZA,SGLI_SZA_PL)))
                                    SZA_Matching_PL = abs(AHI_SZA - SGLI_SZA_PL)
#                                     print(SZA_Matching_PL)
#                                     if SZA_Matching_PL <= 0.02 and SZA_Matching_PL != 0:
                                    if SZA_Matching_PL <= 10:


                                        
                                        
                                        print('PL')
                                        print('AHI:{}'.format(date_new + TIME))
                                        print('SGLI:{}'.format(date))
                                        print("AHI_RAA",AHI_RAA,"SGLI_RAA",SGLI_RAA_PL)
                                        print("AHI_VZA",AHI_VZA,"SGLI_VZA",SGLI_VZA_PL)
                                        print("AHI_SZA",AHI_SZA,"SGLI_SZA",SGLI_SZA_PL)
                                        print("AHI_SAA",AHI_SAA,"SGLI_SAA",SGLI_SAA_PL)
                                        print("AHI_VAA",AHI_VAA,"SGLI_VAA",SGLI_VAA_PL)
                                        MATCH_RESULT.append('PL')
                                        MATCH_RESULT.append('AHI:{}'.format(date_new + TIME))
                                        MATCH_RESULT.append('SGLI:{}'.format(date))
                                        MATCH_RESULT.append("AHI_RAA:{} SGLI_RAA:{}".format(AHI_RAA,SGLI_RAA_PL))
                                        MATCH_RESULT.append("AHI_VZA:{} SGLI_VZA:{}".format(AHI_VZA,SGLI_VZA_PL))
                                        MATCH_RESULT.append("AHI_SZA:{} SGLI_SZA:{}".format(AHI_SZA,SGLI_SZA_PL))
                                        
                                                           



    
# with open("{}_{}_PL.txt".format(site_name,YYYY),"w") as f:
#     f.writelines('\n'.join(MATCH_RESULT))              

20210101
20210102


/home/liwei/.local/lib/python3.8/site-packages/skimage/measure/block.py:89: RuntimeWarning: Mean of empty slice
  return func(blocked, axis=tuple(range(image.ndim, blocked.ndim)),
/tmp/ipykernel_1178077/3354541270.py:27: RuntimeWarning: Mean of empty slice
  SGLI_VZA_PL = np.nanmean(SGLI_VZA_PL)


20210103
20210104
20210105
20210106


/home/liwei/.local/lib/python3.8/site-packages/skimage/measure/block.py:89: RuntimeWarning: Mean of empty slice
  return func(blocked, axis=tuple(range(image.ndim, blocked.ndim)),
/tmp/ipykernel_1178077/3354541270.py:27: RuntimeWarning: Mean of empty slice
  SGLI_VZA_PL = np.nanmean(SGLI_VZA_PL)
/home/liwei/.local/lib/python3.8/site-packages/skimage/measure/block.py:89: RuntimeWarning: Mean of empty slice
  return func(blocked, axis=tuple(range(image.ndim, blocked.ndim)),
/tmp/ipykernel_1178077/3354541270.py:27: RuntimeWarning: Mean of empty slice
  SGLI_VZA_PL = np.nanmean(SGLI_VZA_PL)


20210107
20210108
20210109
20210110


...

20210221
20210222
20210223
20210224


/home/liwei/.local/lib/python3.8/site-packages/skimage/measure/block.py:89: RuntimeWarning: Mean of empty slice
  return func(blocked, axis=tuple(range(image.ndim, blocked.ndim)),
/tmp/ipykernel_1178077/3354541270.py:27: RuntimeWarning: Mean of empty slice
  SGLI_VZA_PL = np.nanmean(SGLI_VZA_PL)
/home/liwei/.local/lib/python3.8/site-packages/skimage/measure/block.py:89: RuntimeWarning: Mean of empty slice
  return func(blocked, axis=tuple(range(image.ndim, blocked.ndim)),
/tmp/ipykernel_1178077/3354541270.py:27: RuntimeWarning: Mean of empty slice
  SGLI_VZA_PL = np.nanmean(SGLI_VZA_PL)


20210225
20210226
20210227
20210228


/home/liwei/.local/lib/python3.8/site-packages/skimage/measure/block.py:89: RuntimeWarning: Mean of empty slice
  return func(blocked, axis=tuple(range(image.ndim, blocked.ndim)),
/tmp/ipykernel_1178077/3354541270.py:27: RuntimeWarning: Mean of empty slice
  SGLI_VZA_PL = np.nanmean(SGLI_VZA_PL)
/home/liwei/.local/lib/python3.8/site-packages/skimage/measure/block.py:89: RuntimeWarning: Mean of empty slice
  return func(blocked, axis=tuple(range(image.ndim, blocked.ndim)),
/tmp/ipykernel_1178077/3354541270.py:27: RuntimeWarning: Mean of empty slice
  SGLI_VZA_PL = np.nanmean(SGLI_VZA_PL)


20210301


In [ ]:
with open("{}_{}_PL.txt".format(site_name,YYYY),"w") as f:
    f.writelines('\n'.join(MATCH_RESULT)) 

In [18]:
MATCH_RESULT

['PL',
 'AHI:202101132200',
 'SGLI:20210113',
 'AHI_RAA:107.52500505200896 SGLI_RAA:99.89348749999998',
 'AHI_VZA:58.55299028268415 SGLI_VZA:57.19599375',
 'AHI_SZA:39.988929748535156 SGLI_SZA:41.15301875',
 'PL',
 'AHI:202102012140',
 'SGLI:20210201',
 'AHI_RAA:108.86403245679412 SGLI_RAA:100.09074374999997',
 'AHI_VZA:58.55299028268415 SGLI_VZA:58.30483125000001',
 'AHI_SZA:47.301902770996094 SGLI_SZA:45.743599999999994',
 'PL',
 'AHI:202102012150',
 'SGLI:20210201',
 'AHI_RAA:106.50625200025115 SGLI_RAA:100.09074374999997',
 'AHI_VZA:58.55299028268415 SGLI_VZA:58.30483125000001',
 'AHI_SZA:45.62739944458008 SGLI_SZA:45.743599999999994',
 'PL',
 'AHI:202102012200',
 'SGLI:20210201',
 'AHI_RAA:104.04027909986053 SGLI_RAA:100.09074374999997',
 'AHI_VZA:58.55299028268415 SGLI_VZA:58.30483125000001',
 'AHI_SZA:43.98129653930664 SGLI_SZA:45.743599999999994',
 'PL',
 'AHI:202102162140',
 'SGLI:20210216',
 'AHI_RAA:104.54856462232146 SGLI_RAA:107.08823124999999',
 'AHI_VZA:58.55299028268415

In [14]:
for m in range(len(MM)):
    s = month_size(MM[m],YYYY)
    for d in range(s):
        date = YYYY+MM[m]+DD[d]
        
#         SGLI file name
        SGLI_VZA_filename = 'GC1SG1_{}D01D_T{}_L2SG_RSRFQ_3001_Sensor_zenith_PL.tif'.format(date,tile)
        SGLI_VAA_filename = 'GC1SG1_{}D01D_T{}_L2SG_RSRFQ_3001_Sensor_azimuth_PL.tif'.format(date,tile)
        SGLI_SZA_filename = 'GC1SG1_{}D01D_T{}_L2SG_RSRFQ_3001_Solar_zenith_PL.tif'.format(date,tile)
        SGLI_SAA_filename = 'GC1SG1_{}D01D_T{}_L2SG_RSRFQ_3001_Solar_azimuth_PL.tif'.format(date,tile)
        
        
#         Reporjection file exist
        if not os.path.exists(reporjection_file + SGLI_VZA_filename):
            
#             print('Start SGLI Download and Reporjection.')
            fn = download_tile(date,tile_file,tile)
            if fn == 'n':
                continue
            for b in band:
                reporjection_GEO( tile_file+fn , b , reporjection_file)
#             print('Reporjection Finsihed!')
             
        if os.path.exists(reporjection_file + SGLI_VZA_filename):       
            SGLI_VZA_PL = SGLI_2_AHI_GEO(reporjection_file + SGLI_VZA_filename)
            SGLI_VZA_PL = np.nanmean(SGLI_VZA_PL)
            VZA_Matching_PL = abs(AHI_VZA - SGLI_VZA_PL)
            
            
            
#             VZA_Matching_PL = 1 - math.cos(math.radians(max(AHI_VZA,SGLI_VZA_PL))) / math.cos(math.radians(min(AHI_VZA,SGLI_VZA_PL)))
    #         print("AHI_VZA:",AHI_VZA,"SGLI_VZA:",SGLI_VZA_PL)
#             print(SGLI_VZA_PL)   
            
#             if VZA_Matching_PL <= 0.02 and VZA_Matching_PL!= 0:
#                 print (date)
                
    #             print("AHI_VZA:",AHI_VZA,"SGLI_VZA:",SGLI_VZA_PL)
        
            if VZA_Matching_PL <= 10:
    #             For loop for min
                print(date)

                print("AHI_VZA:",AHI_VZA,"SGLI_VZA:",SGLI_VZA_PL)

                
                SGLI_VAA_PL = SGLI_2_AHI_GEO(reporjection_file + SGLI_VAA_filename)
                SGLI_SAA_PL = SGLI_2_AHI_GEO(reporjection_file + SGLI_SAA_filename)
                SGLI_VAA_PL = np.where(SGLI_VAA_PL<0,SGLI_VAA_PL+360,SGLI_VAA_PL)
                SGLI_SAA_PL = np.where(SGLI_SAA_PL<0,SGLI_SAA_PL+360,SGLI_SAA_PL)
                SGLI_VAA_PL = np.nanmean(SGLI_VAA_PL)
                SGLI_SAA_PL = np.nanmean(SGLI_SAA_PL)
                SGLI_RAA_PL = abs(SGLI_SAA_PL - SGLI_VAA_PL)
#                 print("SGLI_RAA_PL:",SGLI_VAA_PL)

#                 print("SGLI_VAA_PL:",SGLI_VAA_PL,"SGLI_SAA_PL:",SGLI_SAA_PL)
                

/home/liwei/.local/lib/python3.8/site-packages/skimage/measure/block.py:89: RuntimeWarning: Mean of empty slice
  return func(blocked, axis=tuple(range(image.ndim, blocked.ndim)),
/tmp/ipykernel_1178077/2516273744.py:26: RuntimeWarning: Mean of empty slice
  SGLI_VZA_PL = np.nanmean(SGLI_VZA_PL)
/home/liwei/.local/lib/python3.8/site-packages/skimage/measure/block.py:89: RuntimeWarning: Mean of empty slice
  return func(blocked, axis=tuple(range(image.ndim, blocked.ndim)),
/tmp/ipykernel_1178077/2516273744.py:26: RuntimeWarning: Mean of empty slice
  SGLI_VZA_PL = np.nanmean(SGLI_VZA_PL)
/home/liwei/.local/lib/python3.8/site-packages/skimage/measure/block.py:89: RuntimeWarning: Mean of empty slice
  return func(blocked, axis=tuple(range(image.ndim, blocked.ndim)),
/tmp/ipykernel_1178077/2516273744.py:26: RuntimeWarning: Mean of empty slice
  SGLI_VZA_PL = np.nanmean(SGLI_VZA_PL)


20210103
AHI_VZA: 54.216307412290064 SGLI_VZA: 53.99291875
SGLI_VAA_PL: 354.50085 SGLI_SAA_PL: 151.88093125
20210104
AHI_VZA: 54.216307412290064 SGLI_VZA: 57.373537500000005
SGLI_VAA_PL: 35.22281875 SGLI_SAA_PL: 144.5920125
20210107
AHI_VZA: 54.216307412290064 SGLI_VZA: 52.79876875
SGLI_VAA_PL: 3.949375 SGLI_SAA_PL: 149.35374375
20210108
AHI_VZA: 54.216307412290064 SGLI_VZA: 60.35158125
SGLI_VAA_PL: 41.69275625 SGLI_SAA_PL: 142.28196875
20210111
AHI_VZA: 54.216307412290064 SGLI_VZA: 52.79556875
SGLI_VAA_PL: 13.860456250000002 SGLI_SAA_PL: 146.81609999999998
20210114
AHI_VZA: 54.216307412290064 SGLI_VZA: 57.2992
SGLI_VAA_PL: 342.88199999999995 SGLI_SAA_PL: 151.89537500000003
20210115
AHI_VZA: 54.216307412290064 SGLI_VZA: 53.997181250000004


/home/liwei/.local/lib/python3.8/site-packages/skimage/measure/block.py:89: RuntimeWarning: Mean of empty slice
  return func(blocked, axis=tuple(range(image.ndim, blocked.ndim)),
/tmp/ipykernel_1178077/2516273744.py:26: RuntimeWarning: Mean of empty slice
  SGLI_VZA_PL = np.nanmean(SGLI_VZA_PL)
/home/liwei/.local/lib/python3.8/site-packages/skimage/measure/block.py:89: RuntimeWarning: Mean of empty slice
  return func(blocked, axis=tuple(range(image.ndim, blocked.ndim)),
/tmp/ipykernel_1178077/2516273744.py:26: RuntimeWarning: Mean of empty slice
  SGLI_VZA_PL = np.nanmean(SGLI_VZA_PL)
/home/liwei/.local/lib/python3.8/site-packages/skimage/measure/block.py:89: RuntimeWarning: Mean of empty slice
  return func(blocked, axis=tuple(range(image.ndim, blocked.ndim)),
/tmp/ipykernel_1178077/2516273744.py:26: RuntimeWarning: Mean of empty slice
  SGLI_VZA_PL = np.nanmean(SGLI_VZA_PL)
/home/liwei/.local/lib/python3.8/site-packages/skimage/measure/block.py:89: RuntimeWarning: Mean of empty sli

SGLI_VAA_PL: 23.38821875 SGLI_SAA_PL: 144.26715000000002
20210118
AHI_VZA: 54.216307412290064 SGLI_VZA: 54.8522875
SGLI_VAA_PL: 350.61655625 SGLI_SAA_PL: 149.1455125
20210119
AHI_VZA: 54.216307412290064 SGLI_VZA: 56.175575
SGLI_VAA_PL: 31.871443749999997 SGLI_SAA_PL: 141.70731250000003
20210122
AHI_VZA: 54.216307412290064 SGLI_VZA: 53.195062500000006
SGLI_VAA_PL: 356.94 SGLI_SAA_PL: 146.38073125
20210123
AHI_VZA: 54.216307412290064 SGLI_VZA: 58.98830625000001
SGLI_VAA_PL: 38.978412500000005 SGLI_SAA_PL: 139.13871875
20210126
AHI_VZA: 54.216307412290064 SGLI_VZA: 52.6419125
SGLI_VAA_PL: 9.443287500000002 SGLI_SAA_PL: 143.60061249999998
20210129
AHI_VZA: 54.216307412290064 SGLI_VZA: 58.5459


/home/liwei/.local/lib/python3.8/site-packages/skimage/measure/block.py:89: RuntimeWarning: Mean of empty slice
  return func(blocked, axis=tuple(range(image.ndim, blocked.ndim)),
/tmp/ipykernel_1178077/2516273744.py:26: RuntimeWarning: Mean of empty slice
  SGLI_VZA_PL = np.nanmean(SGLI_VZA_PL)
/home/liwei/.local/lib/python3.8/site-packages/skimage/measure/block.py:89: RuntimeWarning: Mean of empty slice
  return func(blocked, axis=tuple(range(image.ndim, blocked.ndim)),
/tmp/ipykernel_1178077/2516273744.py:26: RuntimeWarning: Mean of empty slice
  SGLI_VZA_PL = np.nanmean(SGLI_VZA_PL)
/home/liwei/.local/lib/python3.8/site-packages/skimage/measure/block.py:89: RuntimeWarning: Mean of empty slice
  return func(blocked, axis=tuple(range(image.ndim, blocked.ndim)),
/tmp/ipykernel_1178077/2516273744.py:26: RuntimeWarning: Mean of empty slice
  SGLI_VZA_PL = np.nanmean(SGLI_VZA_PL)
/home/liwei/.local/lib/python3.8/site-packages/skimage/measure/block.py:89: RuntimeWarning: Mean of empty sli

...

/home/liwei/.local/lib/python3.8/site-packages/skimage/measure/block.py:89: RuntimeWarning: Mean of empty slice
  return func(blocked, axis=tuple(range(image.ndim, blocked.ndim)),
/tmp/ipykernel_1178077/2516273744.py:26: RuntimeWarning: Mean of empty slice
  SGLI_VZA_PL = np.nanmean(SGLI_VZA_PL)
/home/liwei/.local/lib/python3.8/site-packages/skimage/measure/block.py:89: RuntimeWarning: Mean of empty slice
  return func(blocked, axis=tuple(range(image.ndim, blocked.ndim)),
/tmp/ipykernel_1178077/2516273744.py:26: RuntimeWarning: Mean of empty slice
  SGLI_VZA_PL = np.nanmean(SGLI_VZA_PL)
/home/liwei/.local/lib/python3.8/site-packages/skimage/measure/block.py:89: RuntimeWarning: Mean of empty slice
  return func(blocked, axis=tuple(range(image.ndim, blocked.ndim)),
/tmp/ipykernel_1178077/2516273744.py:26: RuntimeWarning: Mean of empty slice
  SGLI_VZA_PL = np.nanmean(SGLI_VZA_PL)
/home/liwei/.local/lib/python3.8/site-packages/skimage/measure/block.py:89: RuntimeWarning: Mean of empty sli

20210612
AHI_VZA: 54.216307412290064 SGLI_VZA: 59.431482558139535
SGLI_VAA_PL: 155.34322674418607 SGLI_SAA_PL: 71.86702519379845
20210614
AHI_VZA: 54.216307412290064 SGLI_VZA: 61.332418749999995
SGLI_VAA_PL: 226.73176875 SGLI_SAA_PL: 64.99622500000001
20210615
AHI_VZA: 54.216307412290064 SGLI_VZA: 52.85424375
SGLI_VAA_PL: 186.82679375 SGLI_SAA_PL: 69.6725
20210618
AHI_VZA: 54.216307412290064 SGLI_VZA: 58.38483125
SGLI_VAA_PL: 220.02006874999998 SGLI_SAA_PL: 66.16684375
20210619
AHI_VZA: 54.216307412290064 SGLI_VZA: 53.66670625
SGLI_VAA_PL: 176.5992875 SGLI_SAA_PL: 70.1708


/home/liwei/.local/lib/python3.8/site-packages/skimage/measure/block.py:89: RuntimeWarning: Mean of empty slice
  return func(blocked, axis=tuple(range(image.ndim, blocked.ndim)),
/tmp/ipykernel_1178077/2516273744.py:26: RuntimeWarning: Mean of empty slice
  SGLI_VZA_PL = np.nanmean(SGLI_VZA_PL)
/home/liwei/.local/lib/python3.8/site-packages/skimage/measure/block.py:89: RuntimeWarning: Mean of empty slice
  return func(blocked, axis=tuple(range(image.ndim, blocked.ndim)),
/tmp/ipykernel_1178077/2516273744.py:26: RuntimeWarning: Mean of empty slice
  SGLI_VZA_PL = np.nanmean(SGLI_VZA_PL)
/home/liwei/.local/lib/python3.8/site-packages/skimage/measure/block.py:89: RuntimeWarning: Mean of empty slice
  return func(blocked, axis=tuple(range(image.ndim, blocked.ndim)),
/tmp/ipykernel_1178077/2516273744.py:26: RuntimeWarning: Mean of empty slice
  SGLI_VZA_PL = np.nanmean(SGLI_VZA_PL)


20210622
AHI_VZA: 54.216307412290064 SGLI_VZA: 55.8454875
SGLI_VAA_PL: 212.20592499999998 SGLI_SAA_PL: 67.3788625
20210623
AHI_VZA: 54.216307412290064 SGLI_VZA: 55.417575
SGLI_VAA_PL: 167.47800625000002 SGLI_SAA_PL: 70.79876250000001
20210626
AHI_VZA: 54.216307412290064 SGLI_VZA: 53.97110000000001
SGLI_VAA_PL: 203.39383750000002 SGLI_SAA_PL: 68.65115
20210627
AHI_VZA: 54.216307412290064 SGLI_VZA: 57.78423750000001
SGLI_VAA_PL: 159.62659375 SGLI_SAA_PL: 71.55321250000001
20210630
AHI_VZA: 54.216307412290064 SGLI_VZA: 52.95918125
SGLI_VAA_PL: 193.71415000000002 SGLI_SAA_PL: 70.002075


/home/liwei/.local/lib/python3.8/site-packages/skimage/measure/block.py:89: RuntimeWarning: Mean of empty slice
  return func(blocked, axis=tuple(range(image.ndim, blocked.ndim)),
/tmp/ipykernel_1178077/2516273744.py:26: RuntimeWarning: Mean of empty slice
  SGLI_VZA_PL = np.nanmean(SGLI_VZA_PL)
/home/liwei/.local/lib/python3.8/site-packages/skimage/measure/block.py:89: RuntimeWarning: Mean of empty slice
  return func(blocked, axis=tuple(range(image.ndim, blocked.ndim)),
/tmp/ipykernel_1178077/2516273744.py:26: RuntimeWarning: Mean of empty slice
  SGLI_VZA_PL = np.nanmean(SGLI_VZA_PL)
